## Loading required libraries 📔 and dataset 🗄️🔽

In [ ]:
# installing required libraries
# numerapi, for facilitating data download and predictions uploading

!pip install numerapi
!pip install xgboost
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import scipy
from tqdm import tqdm
from xgboost import XGBRegressor
import gc
import json
from pathlib import Path
from scipy.stats import skew

from numerapi import NumerAPI
import utils
# from utils import (
#     save_model,
#     load_model,
#     neutralize,
#     get_biggest_change_features,
#     validation_metrics,
#     ERA_COL,
#     DATA_TYPE_COL,
#     TARGET_COL,
#     EXAMPLE_PREDS_COL
# )

In [ ]:
# download all the things

napi = NumerAPI()

current_round = napi.get_current_round()

# Tournament data changes every week so we specify the round in their name. Training
# and validation data only change periodically, so no need to download them every time.
print('Downloading dataset files...')

Path("./v4").mkdir(parents=False, exist_ok=True)
napi.download_dataset("v4/train.parquet")
napi.download_dataset("v4/validation.parquet")
napi.download_dataset("v4/live.parquet", f"v4/live_{current_round}.parquet")
napi.download_dataset("v4/validation_example_preds.parquet")
napi.download_dataset("v4/features.json")

v4/train.parquet: 1.16GB [00:25, 45.6MB/s]                            
v4/validation.parquet: 1.18GB [00:26, 45.5MB/s]                            
v4/live_467.parquet: 3.39MB [00:00, 7.73MB/s]                           
v4/validation_example_preds.parquet: 58.3MB [00:01, 30.4MB/s]                            
v4/features.json: 562kB [00:00, 2.17MB/s]                           


'v4/features.json'

## Build Feature Set

---



In [ ]:
ERA_COL = "era"
TARGET_COL = "target_nomi_v4_20"
DATA_TYPE_COL = "data_type"
EXAMPLE_PREDS_COL = "example_preds"

MODEL_FOLDER = "models"

In [ ]:
print('Reading minimal training data')
# read the feature metadata and get a feature set (or all the features)
with open("v4/features.json", "r") as f:
    feature_metadata = json.load(f)
# features = list(feature_metadata["feature_stats"].keys()) # get all the features
features = feature_metadata["feature_sets"]["small"] # get the small feature set
# features = feature_metadata["feature_sets"]["medium"] # get the medium feature set
# read in just those features along with era and target columns
read_columns = features + [ERA_COL, DATA_TYPE_COL, TARGET_COL]

Reading minimal training data


## Loading and exploring dataset into memory 🖥️

In [ ]:
# note: sometimes when trying to read the downloaded data you get an error about invalid magic parquet bytes...
# if so, delete the file and rerun the napi.download_dataset to fix the corrupted file
training_data = pd.read_parquet('v4/train.parquet',
                                columns=read_columns)
validation_data = pd.read_parquet('v4/validation.parquet',
                                  columns=read_columns)
live_data = pd.read_parquet(f'v4/live_{current_round}.parquet',
                                  columns=read_columns)

In [ ]:
# pare down the number of eras to every 4th era
every_4th_era = training_data[ERA_COL].unique()[::4]
training_data = training_data[training_data[ERA_COL].isin(every_4th_era)]

In [ ]:
training_data.head()

,feature_bijou_penetrant_syringa,feature_burning_phrygian_axinomancy,feature_censorial_leachier_rickshaw,feature_coastal_edible_whang,feature_coraciiform_sciurine_reef,feature_corporatist_seborrheic_hopi,feature_cyclopedic_maestoso_daguerreotypist,feature_distressed_bloated_disquietude,feature_ecstatic_foundational_crinoidea,feature_elaborate_intimate_bor,...,feature_undrilled_wheezier_countermand,feature_unpainted_censual_pinacoid,feature_unreproved_cultish_glioma,feature_unsizable_ancestral_collocutor,feature_unsustaining_chewier_adnoun,feature_unswaddled_inenarrable_goody,feature_unventilated_sollar_bason,era,data_type,target_nomi_v4_20
id,,,,,,,,,,,,,,,,,,,,,
n003bba8a98662e4,0.00,0.00,0.50,0.25,0.50,1.00,0.50,0.50,0.00,0.50,...,0.75,1.0,1.00,0.5,0.25,0.25,0.00,0001,train,0.25
n003bee128c2fcfc,0.50,0.75,0.75,0.75,0.50,0.25,0.50,0.75,1.00,0.75,...,0.25,1.0,0.25,0.5,1.00,0.75,0.25,0001,train,0.75
n0048ac83aff7194,0.25,0.25,1.00,0.75,1.00,0.75,0.75,0.75,0.75,0.75,...,0.75,0.0,0.25,1.0,1.00,0.75,1.00,0001,train,0.50
n00691bec80d3e02,0.75,0.75,0.00,0.00,0.75,0.25,0.25,0.00,0.00,0.00,...,0.50,1.0,0.00,0.5,0.00,0.50,0.75,0001,train,0.75
n00b8720a2fdc4f2,0.00,0.00,0.50,0.25,0.00,1.00,0.25,0.25,0.75,0.25,...,0.25,1.0,0.00,0.0,0.25,1.00,0.00,0001,train,0.75


In [ ]:
live_data.head()

,feature_bijou_penetrant_syringa,feature_burning_phrygian_axinomancy,feature_censorial_leachier_rickshaw,feature_coastal_edible_whang,feature_coraciiform_sciurine_reef,feature_corporatist_seborrheic_hopi,feature_cyclopedic_maestoso_daguerreotypist,feature_distressed_bloated_disquietude,feature_ecstatic_foundational_crinoidea,feature_elaborate_intimate_bor,...,feature_undrilled_wheezier_countermand,feature_unpainted_censual_pinacoid,feature_unreproved_cultish_glioma,feature_unsizable_ancestral_collocutor,feature_unsustaining_chewier_adnoun,feature_unswaddled_inenarrable_goody,feature_unventilated_sollar_bason,era,data_type,target_nomi_v4_20
id,,,,,,,,,,,,,,,,,,,,,
n00032303e32b0a2,1.00,1.00,0.75,0.50,0.50,1.00,0.00,1.00,0.00,1.00,...,0.00,0.00,0.50,0.50,0.50,0.75,1.00,X,live,NaN
n0012a90672341dc,1.00,1.00,1.00,0.75,0.75,0.75,0.25,1.00,1.00,1.00,...,0.50,0.50,0.50,0.75,1.00,0.00,0.25,X,live,NaN
n0015887848eec83,0.25,1.00,1.00,1.00,0.00,0.00,0.25,1.00,0.00,1.00,...,0.00,0.25,1.00,0.00,1.00,0.25,0.00,X,live,NaN
n001dc25eee8945c,1.00,0.75,0.75,1.00,1.00,1.00,0.25,0.25,0.75,0.25,...,1.00,0.50,0.00,1.00,0.75,0.50,0.75,X,live,NaN
n00219c6d7dcd08f,0.00,0.00,0.00,0.00,0.00,0.75,0.75,0.75,0.00,0.50,...,0.25,0.00,0.75,0.00,0.00,1.00,0.25,X,live,NaN


In [ ]:
# getting the per era correlation of each feature vs the target
all_feature_corrs = training_data.groupby(ERA_COL).apply(
    lambda era: era[features].corrwith(era[TARGET_COL])
)

In [ ]:
# helper function

def get_biggest_change_features(corrs, n):
    all_eras = corrs.index.sort_values()
    h1_eras = all_eras[: len(all_eras) // 2]
    h2_eras = all_eras[len(all_eras) // 2 :]

    h1_corr_means = corrs.loc[h1_eras, :].mean()
    h2_corr_means = corrs.loc[h2_eras, :].mean()

    corr_diffs = h2_corr_means - h1_corr_means
    worst_n = corr_diffs.abs().sort_values(ascending=False).head(n).index.tolist()
    return worst_n

In [ ]:
# find the riskiest features by comparing their correlation vs
# the target in each half of training data; we'll use these later
riskiest_features = get_biggest_change_features(all_feature_corrs, 50)

In [ ]:
# "garbage collection" (gc) gets rid of unused data and frees up memory
gc.collect()

0

## Training our model 🤖⚙️

This is where most of tweaking will happen. You can add more model in your pipeline simply by changing your model and data pipeline suited for that architecture.

In [ ]:
# Helper Functions

def save_prediction(df, name):
    try:
        Path(PREDICTION_FILES_FOLDER).mkdir(exist_ok=True, parents=True)
    except Exception as ex:
        pass
    df.to_csv(f"{PREDICTION_FILES_FOLDER}/{name}.csv", index=True)


def save_model(model, name):
    try:
        Path(MODEL_FOLDER).mkdir(exist_ok=True, parents=True)
    except Exception as ex:
        pass
    pd.to_pickle(model, f"{MODEL_FOLDER}/{name}.pkl")


def load_model(name):
    path = Path(f"{MODEL_FOLDER}/{name}.pkl")
    if path.is_file():
        model = pd.read_pickle(f"{MODEL_FOLDER}/{name}.pkl")
    else:
        model = False
    return model

In [ ]:
model_name = f"model_target"
print(f"Checking for existing model '{model_name}'")
model = load_model(model_name)
if not model:
    print(f"model not found, creating new one")
    params = {"n_estimators": 2000,
              "learning_rate": 0.01,
              "max_depth": 5,
              "colsample_bytree": 0.1}

    model = XGBRegressor(**params)

    # train on all of train and save the model so we don't have to train next time
    model.fit(training_data.filter(like='feature_', axis='columns'),
              training_data[TARGET_COL])
    print(f"saving new model: {model_name}")
    save_model(model, model_name)

Checking for existing model 'model_target'
model not found, creating new one
saving new model: model_target


In [ ]:
gc.collect()

37

In [ ]:
nans_per_col = live_data[live_data["data_type"] == "live"][features].isna().sum()

# check for nans and fill nans
if nans_per_col.any():
    total_rows = len(live_data[live_data["data_type"] == "live"])
    print(f"Number of nans per column this week: {nans_per_col[nans_per_col > 0]}")
    print(f"out of {total_rows} total rows")
    print(f"filling nans with 0.5")
    live_data.loc[:, features] = live_data.loc[:, features].fillna(0.5)

else:
    print("No nans in the features this week!")

No nans in the features this week!


In [ ]:
# double check the feature that the model expects vs what is available to prevent our
# pipeline from failing if Numerai adds more data and we don't have time to retrain!
model_expected_features = model.get_booster().get_score(importance_type='gain')
if set(model_expected_features) != set(features):
    print(f"New features are available! Might want to retrain model {model_name}.")
validation_data.loc[:, f"preds_{model_name}"] = model.predict(
    validation_data.loc[:, model_expected_features])
live_data.loc[:, f"preds_{model_name}"] = model.predict(
    live_data.loc[:, model_expected_features])

<ipython-input-18-65d97dab2657>:7: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  validation_data.loc[:, model_expected_features])
<ipython-input-18-65d97dab2657>:9: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  live_data.loc[:, model_expected_features])


In [ ]:
gc.collect()

0

## Neutralize

In [ ]:
# Helper function

def neutralize(
    df, columns, neutralizers=None, proportion=1.0, normalize=True, era_col="era", verbose=False
):
    if neutralizers is None:
        neutralizers = []
    unique_eras = df[era_col].unique()
    computed = []
    if verbose:
        iterator = tqdm(unique_eras)
    else:
        iterator = unique_eras
    for u in iterator:
        df_era = df[df[era_col] == u]
        scores = df_era[columns].values
        if normalize:
            scores2 = []
            for x in scores.T:
                x = (scipy.stats.rankdata(x, method="ordinal") - 0.5) / len(x)
                x = scipy.stats.norm.ppf(x)
                scores2.append(x)
            scores = np.array(scores2).T
        exposures = df_era[neutralizers].values

        scores -= proportion * exposures.dot(
            np.linalg.pinv(exposures.astype(np.float32), rcond=1e-6).dot(
                scores.astype(np.float32)
            )
        )

        scores /= scores.std(ddof=0)

        computed.append(scores)

    return pd.DataFrame(np.concatenate(computed), columns=columns, index=df.index)

In [ ]:
# neutralize our predictions to the riskiest features
validation_data[f"preds_{model_name}_neutral_riskiest_50"] = neutralize(
    df=validation_data,
    columns=[f"preds_{model_name}"],
    neutralizers=riskiest_features,
    proportion=1.0,
    normalize=True,
    era_col=ERA_COL
)

live_data[f"preds_{model_name}_neutral_riskiest_50"] = neutralize(
    df=live_data,
    columns=[f"preds_{model_name}"],
    neutralizers=riskiest_features,
    proportion=1.0,
    normalize=True,
    era_col=ERA_COL
)

## Predictions. Evaluation. ➡️

In [ ]:
model_to_submit = f"preds_{model_name}_neutral_riskiest_50"

# rename best model to "prediction" and rank from 0 to 1 to meet upload requirements
validation_data["prediction"] = validation_data[model_to_submit].rank(pct=True)
live_data["prediction"] = live_data[model_to_submit].rank(pct=True)
validation_data["prediction"].to_csv(f"validation_predictions_{current_round}.csv")
live_data["prediction"].to_csv(f"live_predictions_{current_round}.csv")

validation_preds = pd.read_parquet('v4/validation_example_preds.parquet')
validation_data[EXAMPLE_PREDS_COL] = validation_preds["prediction"]

In [ ]:
# Helper Functions

def neutralize_series(series, by, proportion=1.0):
    scores = series.values.reshape(-1, 1)
    exposures = by.values.reshape(-1, 1)

    # this line makes series neutral to a constant column so that it's centered and for sure gets corr 0 with exposures
    exposures = np.hstack(
        (exposures, np.array([np.mean(series)] * len(exposures)).reshape(-1, 1))
    )

    correction = proportion * (
        exposures.dot(np.linalg.lstsq(exposures, scores, rcond=None)[0])
    )
    corrected_scores = scores - correction
    neutralized = pd.Series(corrected_scores.ravel(), index=series.index)
    return neutralized


def unif(df):
    x = (df.rank(method="first") - 0.5) / len(df)
    return pd.Series(x, index=df.index)


def get_feature_neutral_mean(
    df, prediction_col, target_col, features_for_neutralization=None
):
    if features_for_neutralization is None:
        features_for_neutralization = [c for c in df.columns if c.startswith("feature")]
    df.loc[:, "neutral_sub"] = neutralize(
        df, [prediction_col], features_for_neutralization
    )[prediction_col]
    scores = (
        df.groupby("era")
        .apply(lambda x: (unif(x["neutral_sub"]).corr(x[target_col])))
        .mean()
    )
    return np.mean(scores)


def get_feature_neutral_mean_tb_era(
    df, prediction_col, target_col, tb, features_for_neutralization=None
):
    if features_for_neutralization is None:
        features_for_neutralization = [c for c in df.columns if c.startswith("feature")]
    temp_df = df.reset_index(
        drop=True
    ).copy()  # Reset index due to use of argsort later
    temp_df.loc[:, "neutral_sub"] = neutralize(
        temp_df, [prediction_col], features_for_neutralization
    )[prediction_col]
    temp_df_argsort = temp_df.loc[:, "neutral_sub"].argsort()
    temp_df_tb_idx = pd.concat([temp_df_argsort.iloc[:tb], temp_df_argsort.iloc[-tb:]])
    temp_df_tb = temp_df.loc[temp_df_tb_idx]
    tb_fnc = unif(temp_df_tb["neutral_sub"]).corr(temp_df_tb[target_col])
    return tb_fnc


def fast_score_by_date(df, columns, target, tb=None, era_col="era"):
    unique_eras = df[era_col].unique()
    computed = []
    for u in unique_eras:
        df_era = df[df[era_col] == u]
        era_pred = np.float64(df_era[columns].values.T)
        era_target = np.float64(df_era[target].values.T)

        if tb is None:
            ccs = np.corrcoef(era_target, era_pred)[0, 1:]
        else:
            tbidx = np.argsort(era_pred, axis=1)
            tbidx = np.concatenate([tbidx[:, :tb], tbidx[:, -tb:]], axis=1)
            ccs = [
                np.corrcoef(era_target[tmpidx], tmppred[tmpidx])[0, 1]
                for tmpidx, tmppred in zip(tbidx, era_pred)
            ]
            ccs = np.array(ccs)

        computed.append(ccs)

    return pd.DataFrame(np.array(computed), columns=columns, index=df[era_col].unique())


def exposure_dissimilarity_per_era(df, prediction_col, example_col, feature_cols=None):
    if feature_cols is None:
        feature_cols = [c for c in df.columns if c.startswith("feature")]
    u = df.loc[:, feature_cols].corrwith(df[prediction_col])
    e = df.loc[:, feature_cols].corrwith(df[example_col])
    return 1 - (np.dot(u, e) / np.dot(e, e))


def validation_metrics(
    validation_data,
    pred_cols,
    example_col,
    fast_mode=False,
    target_col=TARGET_COL,
    features_for_neutralization=None,
):
    validation_stats = pd.DataFrame()
    feature_cols = [c for c in validation_data if c.startswith("feature_")]
    for pred_col in pred_cols:
        # Check the per-era correlations on the validation set (out of sample)
        validation_correlations = validation_data.groupby(ERA_COL).apply(
            lambda d: unif(d[pred_col]).corr(d[target_col])
        )

        mean = validation_correlations.mean()
        std = validation_correlations.std(ddof=0)
        sharpe = mean / std

        validation_stats.loc["mean", pred_col] = mean
        validation_stats.loc["std", pred_col] = std
        validation_stats.loc["sharpe", pred_col] = sharpe

        rolling_max = (
            (validation_correlations + 1)
            .cumprod()
            .rolling(window=9000, min_periods=1)  # arbitrarily large
            .max()
        )
        daily_value = (validation_correlations + 1).cumprod()
        max_drawdown = -((rolling_max - daily_value) / rolling_max).max()
        validation_stats.loc["max_drawdown", pred_col] = max_drawdown

        payout_scores = validation_correlations.clip(-0.25, 0.25)
        payout_daily_value = (payout_scores + 1).cumprod()

        apy = (
            ((payout_daily_value.dropna().iloc[-1]) ** (1 / len(payout_scores)))
            ** 49  # 52 weeks of compounding minus 3 for stake compounding lag
            - 1
        ) * 100

        validation_stats.loc["apy", pred_col] = apy

        if not fast_mode:
            # Check the feature exposure of your validation predictions
            max_per_era = validation_data.groupby(ERA_COL).apply(
                lambda d: d[feature_cols].corrwith(d[pred_col]).abs().max()
            )
            max_feature_exposure = max_per_era.mean()
            validation_stats.loc[
                "max_feature_exposure", pred_col
            ] = max_feature_exposure

            # Check feature neutral mean
            feature_neutral_mean = get_feature_neutral_mean(
                validation_data, pred_col, target_col, features_for_neutralization
            )
            validation_stats.loc[
                "feature_neutral_mean", pred_col
            ] = feature_neutral_mean

            # Check TB200 feature neutral mean
            tb200_feature_neutral_mean_era = validation_data.groupby(ERA_COL).apply(
                lambda df: get_feature_neutral_mean_tb_era(
                    df, pred_col, target_col, 200, features_for_neutralization
                )
            )
            validation_stats.loc[
                "tb200_feature_neutral_mean", pred_col
            ] = tb200_feature_neutral_mean_era.mean()

            # Check top and bottom 200 metrics (TB200)
            tb200_validation_correlations = fast_score_by_date(
                validation_data, [pred_col], target_col, tb=200, era_col=ERA_COL
            )

            tb200_mean = tb200_validation_correlations.mean()[pred_col]
            tb200_std = tb200_validation_correlations.std(ddof=0)[pred_col]
            tb200_sharpe = tb200_mean / tb200_std

            validation_stats.loc["tb200_mean", pred_col] = tb200_mean
            validation_stats.loc["tb200_std", pred_col] = tb200_std
            validation_stats.loc["tb200_sharpe", pred_col] = tb200_sharpe

        # MMC over validation
        mmc_scores = []
        corr_scores = []
        for _, x in validation_data.groupby(ERA_COL):
            series = neutralize_series(unif(x[pred_col]), (x[example_col]))
            mmc_scores.append(np.cov(series, x[target_col])[0, 1] / (0.29**2))
            corr_scores.append(unif(x[pred_col]).corr(x[target_col]))

        val_mmc_mean = np.mean(mmc_scores)
        val_mmc_std = np.std(mmc_scores)
        corr_plus_mmcs = [c + m for c, m in zip(corr_scores, mmc_scores)]
        corr_plus_mmc_sharpe = np.mean(corr_plus_mmcs) / np.std(corr_plus_mmcs)

        validation_stats.loc["mmc_mean", pred_col] = val_mmc_mean
        validation_stats.loc["corr_plus_mmc_sharpe", pred_col] = corr_plus_mmc_sharpe

        # Check correlation with example predictions
        per_era_corrs = validation_data.groupby(ERA_COL).apply(
            lambda d: unif(d[pred_col]).corr(unif(d[example_col]))
        )
        corr_with_example_preds = per_era_corrs.mean()
        validation_stats.loc[
            "corr_with_example_preds", pred_col
        ] = corr_with_example_preds

        # Check exposure dissimilarity per era
        tdf = validation_data.groupby(ERA_COL).apply(
            lambda df: exposure_dissimilarity_per_era(
                df, pred_col, example_col, feature_cols
            )
        )
        validation_stats.loc["exposure_dissimilarity_mean", pred_col] = tdf.mean()

    # .transpose so that stats are columns and the model_name is the row
    return validation_stats.transpose()

In [ ]:
# get some stats about each of our models to compare...
# fast_mode=True so that we skip some of the stats that are slower to calculate
validation_stats = validation_metrics(validation_data, [model_to_submit, f"preds_{model_name}"], example_col=EXAMPLE_PREDS_COL, fast_mode=True, target_col=TARGET_COL)
print(validation_stats[["mean", "sharpe"]].to_markdown())

|                                        |      mean |   sharpe |
|:---------------------------------------|----------:|---------:|
| preds_model_target_neutral_riskiest_50 | 0.0130173 | 0.643508 |
| preds_model_target                     | 0.0175353 | 0.630208 |


In [ ]:
print(f'''
Done! Next steps:
    1. Go to numer.ai/tournament (make sure you have an account)
    2. Submit validation_predictions_{current_round}.csv to the diagnostics tool
    3. Submit tournament_predictions_{current_round}.csv to the "Upload Predictions" button
''')


Done! Next steps:
    1. Go to numer.ai/tournament (make sure you have an account)
    2. Submit validation_predictions_467.csv to the diagnostics tool
    3. Submit tournament_predictions_467.csv to the "Upload Predictions" button

